# Data Science Final Project

In [1]:
# !pip install --user --upgrade nltk
# !pip install --user --upgrade keras
# !pip install --user --upgrade tensorflow
# !pip install --user --upgrade textblob
from matplotlib import pyplot
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from datetime import datetime
import time
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import gc
from dateutil import parser
import string
import keras
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from textblob import TextBlob
from sklearn.linear_model import SGDClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import SpatialDropout1D
from keras.layers import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.utils import pad_sequences


plt.rcParams.update(plt.rcParamsDefault)
%matplotlib inline
pd.set_option('display.max_columns', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tdepa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


https://www.kaggle.com/datasets/kazanova/sentiment140

https://www.analyticsvidhya.com/blog/2021/06/twitter-sentiment-analysis-a-nlp-use-case-for-beginners/

In [2]:
df = pd.read_csv('twitterdata.csv', encoding = "ISO-8859-1")
print('Loaded file.')
df = pd.DataFrame(np.vstack([df.columns, df])) # Moves column names into row 1
df.columns = ['sentiment', 'id', 'date', 'query', 'user', 'tweet'] # Renames columns
df.replace({'sentiment': {4: 1}}, inplace=True) # Replaces all '4's with '1's in column 'sentiment'
df.replace({'sentiment': {0: -1}}, inplace=True) # Replaces all '0's with '-1's in column 'sentiment'
data_types_dict = {
    'sentiment': int,
    'id': float
}
print('Fixed sentiment values.')
# df['unix_time'] = pd.to_datetime(df['date'], format='%a %b %d %H:%M:%S %Z %Y', errors='coerce').astype(int) / 10**9
# df.dropna(subset=['unix_time'], inplace=True)

# df['unix_time'] = df['date'].apply(lambda x: parser.parse(x, tzinfos={"PDT": -7*3600}).timestamp())
# print('Added unix column.')

df = df.astype(data_types_dict)
df.dtypes

Loaded file.
Fixed sentiment values.


sentiment      int32
id           float64
date          object
query         object
user          object
tweet         object
dtype: object

In [3]:
# df = pd.read_csv('twitterdata_unix.csv', encoding = "ISO-8859-1")
# df = df[['sentiment', 'tweet', 'unix_time']]
df = df.sample(frac=1).reset_index(drop=True) # shuffles rows so its not all 1 and then -1
df = df[['sentiment', 'tweet']]
df.head()

,sentiment,tweet
0,1,Watching the laker game at a nigga's house
1,1,@jaanus83 i was listening to gervais podcasts ...
2,1,@leonkay Big Brother's turned you into a doggy...
3,1,5 points with Chris &amp; Matt. Getting some A...
4,1,Tonight is the season finale! - don't miss a v...


In [4]:
s = df.stack().value_counts()
print(s)

1                                                                                                                                800000
-1                                                                                                                               799999
isPlayer Has Died! Sorry                                                                                                            210
good morning                                                                                                                        118
headache                                                                                                                            115
                                                                                                                                  ...  
@leomakkinje hey I'm on vacation, I'm supposed to do things I like  like sitting next to a pc user with my shiny powerbook ;)         1
@MeghnaK Yes, he does   http://bit.ly/dNmPx And 

In [5]:
# for i in range(10):
#     print(df['tweet'][i])
content = df['tweet'][:5000].values.tolist()
labels = df['sentiment'][:5000].values.tolist()
content = [x.strip() for x in content] # Deletes white space before and after
content[:20]

["Watching the laker game at a nigga's house",
 '@jaanus83 i was listening to gervais podcasts not 10 minutes ago',
 "@leonkay Big Brother's turned you into a doggy woof woof  x",
 '5 points with Chris &amp; Matt. Getting some Als and then going to listen to a band',
 "Tonight is the season finale! - don't miss a very BIG CALVIN &amp; GRANT Surprise! ABC Family 8pm @abcfam  (via @GregoryMichael)",
 "Want to go play in the rain but i've only just washed my hair  boooooooo",
 "...fresh out the jaccuzi...its late but i'm starving..bout to cook..steak, mashed potatoes &amp; corn...mmmmm",
 'http://twitpic.com/6bq78 - look my McFly Shoes',
 '@vaxen_var Have both The Natural Way of Farming and One Straw Revolution (first editions). My dog chewed on one',
 'I am a clump of fowl  sunstroke. Death.',
 '@dannieboyTV:  I know. I can maybe fix that though...',
 '@SheilaBrummer Glad to see you twittering again!!!',
 "I am so excited about The Script on Friday now  2 doses of tv featuring them and I

In [6]:
y = np.array(labels, dtype='int8')
y[:20]

array([ 1,  1,  1,  1,  1, -1,  1,  1, -1, -1, -1,  1,  1, -1, -1,  1,  1,
       -1,  1, -1], dtype=int8)

## stop words #1

In [7]:
def full_remove(x, removal_list):
    for w in removal_list:
        x = x.replace(w, ' ')
    return x

## Remove digits
digits = [str(x) for x in range(10)]
remove_digits = [full_remove(x, digits) for x in content]

## Remove punctuation
remove_punc = [full_remove(x, list(string.punctuation)) for x in remove_digits]

## Make everything lower-case and remove any white space
sents_lower = [x.lower() for x in remove_punc]
sents_lower = [x.strip() for x in sents_lower]

## Remove stop words
from nltk.corpus import stopwords
stops = stopwords.words("English")
def removeStopWords(stopWords, txt):
    newtxt = ' '.join([word for word in txt.split() if word not in stopWords])
    return newtxt
content1 = [removeStopWords(stops,x) for x in sents_lower]
content1[:20]

['watching laker game nigga house',
 'jaanus listening gervais podcasts minutes ago',
 'leonkay big brother turned doggy woof woof x',
 'points chris amp matt getting als going listen band',
 'tonight season finale miss big calvin amp grant surprise abc family pm abcfam via gregorymichael',
 'want go play rain washed hair boooooooo',
 'fresh jaccuzi late starving bout cook steak mashed potatoes amp corn mmmmm',
 'http twitpic com bq look mcfly shoes',
 'vaxen var natural way farming one straw revolution first editions dog chewed one',
 'clump fowl sunstroke death',
 'dannieboytv know maybe fix though',
 'sheilabrummer glad see twittering',
 'excited script friday doses tv featuring excited',
 'hybrid get rid used fine started yesterday',
 'cold',
 'saved dollars grocery store coupons',
 'enjoyed dominos rented bolt spirit gran torino also rented terminator salvation easy k',
 'grrr meant',
 'left digsby running overnight apparently went un idle several times got messages people thought

## stops we defined instead

In [8]:
stop_set = ['the', 'a', 'an', 'i', 'he', 'she', 'they', 'to', 'of', 'it', 'from']
content2 = [removeStopWords(stop_set,x) for x in sents_lower]
content2[:20]

['watching laker game at nigga s house',
 'jaanus was listening gervais podcasts not minutes ago',
 'leonkay big brother s turned you into doggy woof woof x',
 'points with chris amp matt getting some als and then going listen band',
 'tonight is season finale don t miss very big calvin amp grant surprise abc family pm abcfam via gregorymichael',
 'want go play in rain but ve only just washed my hair boooooooo',
 'fresh out jaccuzi its late but m starving bout cook steak mashed potatoes amp corn mmmmm',
 'http twitpic com bq look my mcfly shoes',
 'vaxen var have both natural way farming and one straw revolution first editions my dog chewed on one',
 'am clump fowl sunstroke death',
 'dannieboytv know can maybe fix that though',
 'sheilabrummer glad see you twittering again',
 'am so excited about script on friday now doses tv featuring them and m excited',
 'hybrid can t o get rid used be fine only started doing this yesterday',
 'm cold',
 'saved dollars at grocery store with coupons

## stemming

In [9]:
import nltk
def stemporter(words):
    porter = nltk.PorterStemmer()
    new_words = [porter.stem(w) for w in words]
    return new_words
    
def stemlancaster(words):
    porter = nltk.LancasterStemmer()
    new_words = [porter.stem(w) for w in words]
    return new_words    

porter = [stemporter(x.split()) for x in content2]
porter = [" ".join(i) for i in porter]
porter[0:10]

['watch laker game at nigga s hous',
 'jaanu wa listen gervai podcast not minut ago',
 'leonkay big brother s turn you into doggi woof woof x',
 'point with chri amp matt get some al and then go listen band',
 'tonight is season final don t miss veri big calvin amp grant surpris abc famili pm abcfam via gregorymichael',
 'want go play in rain but ve onli just wash my hair boooooooo',
 'fresh out jaccuzi it late but m starv bout cook steak mash potato amp corn mmmmm',
 'http twitpic com bq look my mcfli shoe',
 'vaxen var have both natur way farm and one straw revolut first edit my dog chew on one',
 'am clump fowl sunstrok death']

## vectorizing

In [10]:
vectorizer = CountVectorizer(analyzer = "word", 
                             preprocessor = None, 
                             stop_words =  'english', 
                             max_features = 6000, ngram_range=(1,5))
data_features = vectorizer.fit_transform(content2)
tfidf_transformer = TfidfTransformer()
data_features_tfidf = tfidf_transformer.fit_transform(data_features)
data_mat = data_features_tfidf.toarray()

np.random.seed(0)
test_index = np.append(np.random.choice((np.where(y==-1))[0], 250, replace=False), np.random.choice((np.where(y==1))[0], 250, replace=False))
train_index = list(set(range(len(labels))) - set(test_index))
train_data = data_mat[train_index,]
train_labels = y[train_index]
test_data = data_mat[test_index,]
test_labels = y[test_index]

#Create polarity function and subjectivity function
pol = lambda x: TextBlob(x).sentiment.polarity
sub = lambda x: TextBlob(x).sentiment.subjectivity
pol_list = [pol(x) for x in content2]
sub_list = [sub(x) for x in content2]

In [11]:
for i in range(10):
    print(content2[i], '\t', pol_list[i], sub_list[i])

watching laker game at nigga s house 	 -0.4 0.4
jaanus was listening gervais podcasts not minutes ago 	 0.0 0.0
leonkay big brother s turned you into doggy woof woof x 	 0.0 0.1
points with chris amp matt getting some als and then going listen band 	 0.0 0.0
tonight is season finale don t miss very big calvin amp grant surprise abc family pm abcfam via gregorymichael 	 0.0 0.13
want go play in rain but ve only just washed my hair boooooooo 	 0.0 1.0
fresh out jaccuzi its late but m starving bout cook steak mashed potatoes amp corn mmmmm 	 0.0 0.55
http twitpic com bq look my mcfly shoes 	 0.0 0.0
vaxen var have both natural way farming and one straw revolution first editions my dog chewed on one 	 0.175 0.3666666666666667
am clump fowl sunstroke death 	 0.0 0.0


In [12]:
## Fit logistic classifier on training data
clf = SGDClassifier(loss="log", penalty="none")
clf.fit(train_data, train_labels)
## Pull out the parameters (w,b) of the logistic regression model
w = clf.coef_[0,:]
b = clf.intercept_
## Get predictions on training and test data
preds_train = clf.predict(train_data)
preds_test = clf.predict(test_data)
## Compute errors
errs_train = np.sum((preds_train > 0.0) != (train_labels > 0.0))
errs_test = np.sum((preds_test > 0.0) != (test_labels > 0.0))
print("Training error: ", float(errs_train)/len(train_labels))
print("Test error: ", float(errs_test)/len(test_labels))

Training error:  0.04688888888888889
Test error:  0.348


In [13]:
## Convert vocabulary into a list:
vocab = np.array([z[0] for z in sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])])
## Get indices of sorting w
inds = np.argsort(w)
## Words with large negative values
neg_inds = inds[0:50]
print("Highly negative words: ")
# MB: fixed bug here
print([x for x in list(vocab[neg_inds])])
## Words with large positive values
pos_inds = inds[-49:-1]
print("Highly positive words: ")
print([x for x in list(vocab[pos_inds])])

Highly negative words: 
['sad', 'headache', 'poor', 'scared', 'unfortunately', 'happened', 'ughhhh', 'missing', 'sick', 'understand', 'infection', 'sadly', 'wish', 'upset', 'faster', 'stopped', 'lebron', 'mail', 'didnt', 'weather good', 'monday', 'sucks', 'prayers', 'marco', 'work', 'complete', 'lost', 'shame', 'xxx', 'revision', 'realized', 'bad', 'answer', 'bathroom', 'hahahaha', 'henry', 'rain', 'xox', 'downtown', 'cancelled', 'babysitting', 'ahhhhhh', 'procrastinator', 'website', 'stage', 'oh gosh', 'taken', 'english', 'hun', 'haven']
Highly positive words: 
['mommy', 'called', 'dating', 'did miss', 'meal', 'expect', 'loved', 'purple', 'welcome', 'concert', 'entirely', 'time tweet', 'drink', 'want hear', 'mis', 'hello', 'cookies', 'food', 'dannygokey', 'florida', 'beer', 'excited', 'anytime', 'hillsong', 'ass', 'lot thanks', 'reasons', 'fridge', 'official', 'process', 'fantastic', 'thanks', 'cindy', 'nice', 'ah', 'cute', 'like ll', 'fabulous', 'rocks', 'listening', 'sunshine', 'say

In [14]:
print(clf.predict(vectorizer.transform(["It's a sad movie but very good"])))
print(clf.predict(vectorizer.transform(["Waste of my time"])))
print(clf.predict(vectorizer.transform(["It is not what like"])))
print(clf.predict(vectorizer.transform(["It is not what I m looking for"])))

[-1]
[1]
[1]
[-1]


In [15]:
from sklearn.linear_model import SGDClassifier
svm_clf = SGDClassifier(loss="hinge", penalty='l2')
svm_clf.fit(train_data, train_labels)
svm_preds_test = svm_clf.predict(test_data)
svm_errs_test = np.sum((svm_preds_test > 0.0) != (test_labels > 0.0))
print("Test error: ", float(svm_errs_test)/len(test_labels))

Test error:  0.308


In [16]:
print(svm_clf.predict(vectorizer.transform(["It's a sad movie but very good"])))
print(svm_clf.predict(vectorizer.transform(["Waste of my time"])))
print(svm_clf.predict(vectorizer.transform(["This is not what I like"])))
print(svm_clf.predict(vectorizer.transform(["It is not what I am looking for"])))

[-1]
[-1]
[1]
[-1]


In [17]:
max_review_length = 200
tokenizer = Tokenizer(num_words=10000,  #max no. of unique words to keep
                      filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', 
                      lower=True #convert to lower case
                     )
tokenizer.fit_on_texts(content2)

X = tokenizer.texts_to_sequences(content2)
X = pad_sequences(X, maxlen= max_review_length)
print('Shape of data tensor:', X.shape)

Y=pd.get_dummies(y).values

np.random.seed(0)
test_inds = np.append(np.random.choice((np.where(y==-1))[0], 250, replace=False), np.random.choice((np.where(y==1))[0], 250, replace=False))
train_inds = list(set(range(len(labels))) - set(test_inds))
train_data = X[train_inds,]
train_labels = Y[train_inds]
test_data = X[test_inds,]
test_labels = Y[test_inds]

Shape of data tensor: (5000, 200)


In [18]:
EMBEDDING_DIM = 200
model = Sequential()
model.add(Embedding(10000, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(250, dropout=0.2,return_sequences=True))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 200, 200)          2000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 200, 200)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 200, 250)          451000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               140400    
                                                                 
 dense (Dense)               (None, 2)                 202       
                                                                 
Total params: 2,591,602
Trainable params: 2,591,602
Non-trainable params: 0
______________________________________________

In [ ]:
epochs = 2
batch_size = 40
model.fit(train_data, train_labels, 
          epochs=epochs, 
          batch_size=batch_size,
          validation_split=0.1)

Epoch 1/2
 75/102 [=====================>........] - ETA: 31s - loss: 0.6556 - accuracy: 0.6100

In [ ]:
loss, acc = model.evaluate(test_data, test_labels, verbose=2,
                            batch_size=batch_size)
print(f"Loss: {loss}")
print(f"Validation accuracy: {acc}")

In [ ]:
outcome_labels = ['Negative', 'Positive']
new = ["kill yourself"]
 
def predict_sentiment(text):
    seq = tokenizer.texts_to_sequences(text)
    padded = pad_sequences(seq, maxlen=max_review_length)
    pred = model.predict(padded)
    print("Probability distribution: ", pred)
    print(f"Is this a Positive or Negative message? '{new[0]}'")
    print(outcome_labels[np.argmax(pred)])

predict_sentiment(new)